In [159]:
def davies_meyer(encrypt, l_key: int , l_message: int):
    """
    Arguments :
    encrypt : an encryption function
    l_key : length in bytes of the keys for encrypt
    l_message : length in bytes of the messages for encrypt
    Returns :
    A compression function from messages of length l_key + l_message to
    messages of length l_message , defined by using the Davies - Meyer
    construction
    """
    def compress(message):
        part1 = message[:l_key]
        part2 = message[len(message)-l_message:]
        enc = encrypt(part1, part2)
        return bytearray([a ^ b for a, b in zip(enc, part2)])
    return compress

In [160]:
def pad(message: bytearray, l_block: int) -> bytearray:
    """
    Arguments :
    message : message to be padded
    l_block : length in bytes of the block
    Returns :
    extension of message that includes the length of message
    (in bytes ) in its last block
    """
    largo_original = len(message)
    if len(message) % l_block != 0 or len(message) == 0:
        ceros = l_block - (len(message) % l_block) - 1
        message = message + bytearray([1]) + bytearray(ceros)
    
    message = message + (largo_original % 2**l_block).to_bytes(l_block, "big")
    return message

In [161]:
def merkle_damgard(IV: bytearray, comp, l_block: int):
    """
    Arguments :
    IV: initialization vector for a hash function
    comp : compression function to be used in the Merkle - Damgard
    construction
    l_block : length in bytes of the blocks to be used in the Merkle - Damgard
    construction
    Returns :
    A hash function for messages of arbitrary length , defined by using
    the Merkle - Damgard construction
    """
    def hash_making(message):
        cont = 0
        message = pad(message, l_block)
        while cont < len(message):
            chunk = message[cont:cont+l_block]
            if cont == 0:
                hashed = comp(chunk + IV)
            else:
                hashed = comp(chunk + hashed)
            
            cont += l_block
        
        return hashed      
            
    return hash_making
        

In [162]:
from Crypto.Cipher import AES

def AES_128(key: bytearray, message: bytearray) -> bytearray:
    a = AES.new(key, AES.MODE_ECB)
    return bytearray(a.encrypt(message))

In [163]:
compresion = davies_meyer(AES_128, 16, 16)

In [165]:
hash = merkle_damgard(bytearray(b'1234567890123456'), compresion, 16)

s1 = bytearray(b'Este es un mensaje de prueba para la tarea 2')
s2 = bytearray(b'Este es un mensaje de Prueba para la tarea 2')
s3 = bytearray(b'Un mensaje corto')
s4 = bytearray(b'')

h1 = hash(s1)
h2 = hash(s2)
h3 = hash(s3)
h4 = hash(s4)

print(h1)
print(h2)
print(h3)
print(h4)

bytearray(b'\xe9\xe8\xac\x12\\\xf2\xc8\x16\xceOV\xc5Y.T\xea')
bytearray(b'\xb6\xfb\xc6a\x12\xae\x95\x1f\xda\xc5\x13\xde\x06|Q\x96')
bytearray(b'\xc5\xec\xcdd\xa4(R*\xf0L*QtL\xda\x81')
bytearray(b'F\x9byvI\x974F\xdc\xf9ap\xfb\xa3\xf7\x98')


In [158]:
a = bytearray(b'12')
b = bytearray(b'\x11\x00')
print(bytearray([a ^ b for a, b in zip(a, b)]))

bytearray(b' 2')


In [143]:
bytearray('12','utf-8')

bytearray(b'12')